# Defining a custom XGBoost objective function for time-varying Cox proportional hazards models

Suppose we have $n$ subjects and

$$
    h(t) = h_{0}(t) \cdot e^{\tau(x, t)}
$$

If

* $y_{i}$ is the event indicator for subject $i$ (1 means the subject had an event, 0 means the subject is censored),
* $t_{i}$ is the event or censor time for subject $i$,
* $\hat{y}_{i}(t) = e^{\tau(x, t)}$ is the predicted value for subject $i$ at time $t$ from XGBoost, and
* $R_{i}$ is the set of observations with an event/censor time $t \geq t_{i}$

The Cox proportional hazards model defines the partial likelihood function as

$$
\begin{eqnarray}
             L(y, \hat{y}) & = & \prod_{i = 1}^{n} \left(\frac{\hat{y}_{i}(t_{i})}{\sum_{j \in R_{i}} \hat{y}_{j}(t_{i})}\right)^{y_{i}} \\
    \implies l(y, \hat{y}) & = & \sum_{i = 1}^{n} y_{i} \log \left(\frac{\hat{y}_{i}(t_{i})}{\sum_{j \in R_{i}} \hat{y}_{j}(t_{i})}\right) \\
    \implies l(y, \hat{y}) & = & \sum_{i = 1}^{n} y_{i} \left[ \log{\hat{y}_{i}(t_{i})} - \log\left(\sum_{j \in R_{i}} \hat{y}_{j}(t_{i})\right) \right] \\
\end{eqnarray}
$$

For XGBoost, we need to define the first and second partial derivatives of $l(y, \hat{y})$ with regards to $\hat{y}$. Instead of doing this manually, we'll use ``jax``

In [1]:
import logging
from pathlib import Path
import sys
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

import pandas as pd
from prefect import Flow

from nbaspa.model.tasks import (
    SurvivalData,
    SegmentData,
    CollapseData,
    ConcordanceIndex,
)
from nbaspa.model.tasks.xgboost import FitXGBoost

pd.options.mode.chained_assignment = None

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
df = pd.concat(
    pd.read_csv(fpath, sep="|", dtype={"GAME_ID": str}, index_col=0)
    for fpath in Path("..", "nba-data", "2018-19", "model-data").glob("data_*.csv")
).reset_index(drop=True)

In [3]:
# Initial formatting and segmentation
format_data = SurvivalData(name="Convert input data to range form")
segdata = SegmentData(name="Split data")
# Model training
trained = FitXGBoost(name="Fit XGBoost")

In [4]:
with Flow(name="XGBoost pipeline") as flow:
    # Format and segment the data into train, tune, test
    alldata = format_data(df)
    data = segdata(alldata, splits=[0.8], keys=["train", "test"])
    # Fit the model
    fitted = trained(params={}, train_data=data["train"], verbose_eval=True)

In [ ]:
output = flow.run()

[2021-02-17 07:35:39+0000] INFO - prefect.FlowRunner | Beginning Flow run for 'XGBoost pipeline'
[2021-02-17 07:35:39+0000] INFO - prefect.TaskRunner | Task 'Convert input data to range form': Starting task run...
[2021-02-17 07:36:07+0000] INFO - prefect.TaskRunner | Task 'Convert input data to range form': Finished task run for task with final state: 'Success'
[2021-02-17 07:36:07+0000] INFO - prefect.TaskRunner | Task 'Split data': Starting task run...
[2021-02-17 07:36:07+0000] INFO - prefect.Split data | Setting the seed to 42
[2021-02-17 07:36:07+0000] INFO - prefect.Split data | Dataset ``train`` has 984 games with 116248 rows
[2021-02-17 07:36:07+0000] INFO - prefect.Split data | Dataset ``test`` has 246 games with 29079 rows
[2021-02-17 07:36:07+0000] INFO - prefect.TaskRunner | Task 'Split data': Finished task run for task with final state: 'Success'
[2021-02-17 07:36:07+0000] INFO - prefect.TaskRunner | Task 'GetItem': Starting task run...
[2021-02-17 07:36:07+0000] INFO - p